In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<LANGCHAIN_API_KEY>'

In [2]:
os.environ['OPENAI_API_KEY'] = '<OPENAI_API_KEY>'

In [4]:
os.environ['TAVILY_API_KEY'] ='<TAVILY_API_KEY>'

## Index

In [5]:

### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# Set embeddings
embd = OpenAIEmbeddings()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# load documents
with open("documents/lyft_10k.txt", "r") as text_file:
    doc = text_file.read()


doc =  Document(page_content=doc, metadata={"source": "local"})

In [7]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents([doc])

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embd,
)

# retrive 8 nearest documents 
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})


## LLMs

We use a router to pick between tools.

GPT model decides which tool(s) to call, as well as the how to query them.

In [44]:
### Router

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search",] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


# Preamble
preamble = """You are an expert at routing a user question to a vectorstore or web_search.
The vectorstore contains documents related to the financial performance report of Lyft, the ride sharing company based out of the US.
Use the vectorstore for questions relating to Lyft, the ride sharing company. Otherwise, use web_search."""

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
route_prompt = ChatPromptTemplate.from_messages(
    [("system", preamble),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
response = question_router.invoke(
    {"question": "Who will the Bears draft first in the NFL draft?"}
)
print(response)
response = question_router.invoke({"question": "How will Lyft compete against Uber?"})
print(response)
response = question_router.invoke({"question": "Hi how are you?"})
print(response)
response = question_router.invoke({"question": "What is Lyft's market share in the US?"})
print(response)

datasource='web_search'
datasource='vectorstore'
datasource='web_search'
datasource='vectorstore'


In [9]:
### Retrieval Grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# Prompt
preamble = """You are a grader assessing relevance of a retrieved document to a user question. \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

grade_prompt = ChatPromptTemplate.from_messages(
    [     ("system", preamble),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "How will Lyft compete against Uber?"
docs = retriever.invoke(question)
print(len(docs))
for doc in docs:
    print(doc.page_content)
doc_txt = docs[0].page_content
response = retrieval_grader.invoke({"question": question, "document": doc_txt})
print(response)
print(doc_txt)

8
that is accustomed to on-demand services and has digital-first preferences.  
     • Grow Our Share of Consumers’ Transportation Spend. Lyft’s transportation network is designed to address a wide range of mobility needs. The Lyft
       network spans rideshare, bikes, and scooters and we are well positioned to deliver the best holistic experience to all of our riders and to capture
       significantly more of our market opportunity.                                
     • Deliver Increasing Value to Drivers. We strive to provide drivers that use Lyft with the best possible experience, including access to a variety of
       economic opportunities. For example, through our Express Drive program, drivers can get access to rental cars they can use for ridesharing. We also
       provide drivers with a suite of resources, including access to our on-demand, 24/7 support through our Driver app, to ensure drivers have the resources
       they need before taking the road.                   

In [37]:
### Generate

from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

# Preamble
preamble = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use eleven sentences maximum and keep the answer concise."""

# LLM

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)


question = "How will Lyft compete against Uber?"
# Prompt
def prompt(x):
    return ChatPromptTemplate.from_messages(
        [ ("system", preamble),
            HumanMessage(
                f"Question: {x['question']} \nAnswer: ",
                additional_kwargs={"documents": x["documents"]},
            )
        ]
    )


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents": docs, "question": question})
print(generation)

Lyft competes against Uber by offering competitive pricing, promotions, and discounts to attract customers. They also focus on providing excellent customer service to differentiate themselves from Uber. Lyft invests in driver incentives and bonuses to retain and attract more drivers to their platform. Additionally, Lyft expands its services to new markets and regions to increase its customer base. They also invest in technology and innovation to improve the user experience on their platform. Lyft may collaborate with other companies or form partnerships to enhance their services and offerings. They differentiate themselves by promoting a more socially responsible image compared to Uber. Lyft may also focus on specific niches or target demographics to carve out a unique position in the market. Additionally, Lyft may invest in marketing and advertising campaigns to increase brand awareness and attract more customers. Overall, Lyft's strategy involves a combination of pricing, service qua

In [40]:
### LLM fallback

from langchain_core.output_parsers import StrOutputParser

# Preamble
preamble = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. Use seven sentences maximum and keep the answer concise."""

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Prompt
def prompt(x):
    return ChatPromptTemplate.from_messages(
        
        [("system", preamble),
            HumanMessage(f"Question: {x['question']} \nAnswer: ")]
    )


# Chain
llm_chain = prompt | llm | StrOutputParser()

# Run
question = "Hi how are you? Prove that 1 plus 1 is 2."
generation = llm_chain.invoke({"question": question})
print(generation)

Hello! I'm here to help you with your question. In mathematics, the statement that 1 plus 1 equals 2 is a fundamental principle known as the "principle of identity." This principle is based on the concept of addition, where combining one unit with another unit results in a total of two units. The basic arithmetic operation of addition involves combining two quantities to find their sum. When you add one unit to another unit, the result is always two units. Therefore, based on the principles of arithmetic and mathematical logic, it can be proven that 1 plus 1 equals 2.


In [12]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# Preamble
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)


# Prompt
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeDocuments(binary_score='yes')

In [13]:
### Answer Grader
from typing import Literal

# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: Literal["yes", "no"] = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# Preamble
preamble = """You are a grader assessing whether an answer addresses / resolves a question \n
Give a binary score 'yes' or 'no'. Yes means that the answer resolves the question."""



#LLM with function call 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(GradeAnswer)

#Prompt 
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", preamble),
         ("human", "User question: \n\n {question} \n\n LLM answer: {generation}"),
    ]
)


question = 'What player are the Bears expected to draft first in the 2024 NFL draft?'
generation = ('The Chicago Bears are expected to draft USC quarterback Caleb Williams with ',
 'the first pick in the 2024 NFL draft. Williams was considered the top ',
 'prospect in a draft class loaded with talented quarterbacks.')

answer_grader = answer_prompt | structured_llm_grader
score = answer_grader.invoke({"question": question, "generation": generation})
if score != None:
    print(score)
else:
    print(score)




binary_score='yes'


In [15]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()

## Graph
Capture the flow in as a graph.

### Graph state

In [16]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """|
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

### Graph Flow

In [29]:
from langchain.schema import Document
import pprint


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def llm_fallback(state):
    """
    Generate answer using the LLM w/o vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---LLM Fallback---")
    question = state["question"]
    generation = llm_chain.invoke({"question": question})
    return {"question": question, "generation": generation}


def generate(state):
    """
    Generate answer using the vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    if not isinstance(documents, list):
        documents = [documents]

    # RAG generation
    generation = rag_chain.invoke({"documents": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]


    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})

    # Fallback to LLM or raise error if no decision
    if "tool_calls" not in source.additional_kwargs:
        print("---ROUTE QUESTION TO LLM---")
        return "llm_fallback"
    if len(source.additional_kwargs["tool_calls"]) == 0:
        raise "Router could not decide source"

    # Choose datasource
    datasource = source.additional_kwargs["tool_calls"][0]["function"]["name"]
    if datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    else:
        print("---ROUTE QUESTION TO LLM---")
        return "llm_fallback"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, WEB SEARCH---")
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = 'none' if score is None else score.binary_score
        # pprint.pprint(question)
        # pprint.pprint(generation)
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [30]:

from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # rag
workflow.add_node("llm_fallback", llm_fallback)  # llm

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
        "llm_fallback": "llm_fallback",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",  # Hallucinations: re-generate
        "not useful": "web_search",  # Fails to answer question: fall-back to web-search
        "useful": END,
    },
)
workflow.add_edge("llm_fallback", END)

# Compile
app = workflow.compile()

In [31]:
def generate_answer(question: str):
    # Run
    inputs = {
        "question": question
    }
    for output in app.stream(inputs):
        for key, value in output.items():
            # Node
            pprint.pprint(f"Node '{key}':")
            # Optional: print full state at each node
        pprint.pprint("\n---\n")

    # Final generation
    pprint.pprint(value["generation"])   
    return value["generation"]

In [32]:
generate_answer("What player are the Bears expected to draft first in the 2024 NFL draft?")

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---
"Node 'generate':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION DOES NOT ADDRESS QUESTION---
"Node 'generate':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('The Chicago Bears are expected to draft USC quarterback Caleb Williams with '
 'the No. 1 pick in the 2024 NFL Draft. Williams was widely considered the top '
 'prospect in

'The Chicago Bears are expected to draft USC quarterback Caleb Williams with the No. 1 pick in the 2024 NFL Draft. Williams was widely considered the top prospect in a draft class loaded with talented quarterbacks.'

In [33]:
# Run
generate_answer("How will Lyft compete against Uber?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
How will Lyft compete against Uber?
[Document(metadata={'source': 'local'}, page_content='that is accustomed to on-demand services and has digital-first preferences.  \n     • Grow Our Share of Consumers’ Transportation Spend. Lyft’s transportation network is designed to address a wide range of mobility needs. The Lyft\n       network spans rideshare, bikes, and scooters and we are well positioned to deliver the best holistic experience to all of our riders and to capture\n       significantly more of our market opportunity.                                \n     • Deliver Increasing Value to Drivers. We strive to provide drivers that use Lyft with the best possible experience, including access to a variety of\n       economic opportunities. For example, through our Express Drive program, drivers can get access to rental cars they can use for ridesharin

"Lyft's main ridesharing competitor in the United States and Canada is Uber. Lyft's transportation network spans rideshare, bikes, and scooters, and the company is well-positioned to deliver the best holistic experience to all of its riders and capture significantly more of its market opportunity.\n\nLyft's brand is rooted in hospitality principles: safety, simplicity, reliability, care, and delight. The company believes that users are increasingly choosing services, including a transportation network, based on brand affinity and value alignment. Lyft aims to make it easy for both drivers and riders to choose Lyft every time.\n\nLyft's proprietary technology efficiently matches riders with drivers through advanced dispatching algorithms, providing faster arrival times, localized pricing, and maximum availability. The company also offers drivers access to 24/7 support and earnings tools, as well as education resources and other support to meet their personal goals."

In [34]:
# Run
generate_answer( "What percent of US market share does Lyft have?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
What percent of US market share does Lyft have?
[Document(metadata={'source': 'local'}, page_content='that is accustomed to on-demand services and has digital-first preferences.  \n     • Grow Our Share of Consumers’ Transportation Spend. Lyft’s transportation network is designed to address a wide range of mobility needs. The Lyft\n       network spans rideshare, bikes, and scooters and we are well positioned to deliver the best holistic experience to all of our riders and to capture\n       significantly more of our market opportunity.                                \n     • Deliver Increasing Value to Drivers. We strive to provide drivers that use Lyft with the best possible experience, including access to a variety of\n       economic opportunities. For example, through our Express Drive program, drivers can get access to rental cars they can use fo

"Lyft has a 32% share of the US rideshare market. However, it's worth noting that Lyft's market share in the US was 29% according to Second Measure."

In [35]:
# Run
generate_answer("What is the biggest risk for Lyft?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
What is the biggest risk for Lyft?
[Document(metadata={'source': 'local'}, page_content="claims;                                                                      \n     • our reputation and brand;                                                    \n     • illegal or improper activity of users of our platform;                       \n                                                                                    \n     • the accuracy of background checks on potential or current drivers and our third party providers' ability to effectively conduct such background checks;\n     • changes to our pricing practices;                                            \n     • the growth and development of our network of Light Vehicles and the quality of and supply chain for our Light Vehicles;\n                                                                

'Lyft faces a number of risks, including:\n- Competition from other ridesharing companies, such as Uber\n- The unpredictability of its results of operations and uncertainty regarding the growth of the ridesharing and other markets\n- Its ability to attract and retain qualified drivers and riders\n- Its insurance coverage, the adequacy of its insurance reserves, and the ability of third-party insurance providers to service its auto-related insurance\n- Its ability to successfully develop new offerings on its platform and enhance its existing offerings\n- Its ability to efficiently grow and further develop its network of Light Vehicles, which may not grow as expected or become profitable over time\n- Its ability to manage the complexities associated with its multimodal platform\n- Its ability to offer high-quality user support and to deal with fraud\n- Its ability to effectively manage its Wait & Save offerings\n- Its ability to effectively manage its pricing methodologies\n- Its company

In [36]:
# Run
generate_answer("In what areas is Lyft looking to innovate in the future?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
In what areas is Lyft looking to innovate in the future?
[Document(metadata={'source': 'local'}, page_content='that is accustomed to on-demand services and has digital-first preferences.  \n     • Grow Our Share of Consumers’ Transportation Spend. Lyft’s transportation network is designed to address a wide range of mobility needs. The Lyft\n       network spans rideshare, bikes, and scooters and we are well positioned to deliver the best holistic experience to all of our riders and to capture\n       significantly more of our market opportunity.                                \n     • Deliver Increasing Value to Drivers. We strive to provide drivers that use Lyft with the best possible experience, including access to a variety of\n       economic opportunities. For example, through our Express Drive program, drivers can get access to rental cars they c

"Lyft is looking to innovate in several areas in the future. These include:\n- Investing in marketplace technology to deliver a convenient and high-quality experience to drivers and riders.\n- Growing its share of consumers' transportation spend by delivering the best holistic experience to all riders and capturing more of its market opportunity.\n- Delivering increasing value to drivers by providing access to a variety of economic opportunities and resources.\n- Advocating for social and environmental responsibility by helping drivers transition to electric vehicles, riders take more sustainable transportation modes, and businesses reduce their carbon footprint.\n- Expanding its AI capabilities in its platform, offerings, services and features, including ongoing deployment and improvement of existing machine learning and AI technologies, as well as developing new features using AI technologies."